# Convolutional Autoencoder, MNIST dataset

To do:
- comment
- test

In [1]:
import torch 
from torch.autograd import Variable
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import random
from matplotlib import pyplot as plt

In [2]:
num_epochs = 1
batch_size = 100
learning_rate = 0.001

In [3]:
#dataset for training (download = True to download the dataset for the first time)
train_dataset = torchvision.datasets.MNIST('mnist/',train=True,transform=transforms.ToTensor(), download=False)
#loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#dataset for testing (download = True to download the dataset for the first time)
test_dataset = torchvision.datasets.MNIST('mnist/',train=False,transform=transforms.ToTensor(), download=False)
#loader
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.enc_1 = nn.Sequential(
            nn.Conv2d(1,16, kernel_size = 5, stride = 2),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.enc_2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size = 5, stride = 2),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.enc_3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size = 3, stride = 2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.dec_1 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size = 4, stride = 2),
            nn.ReLU()
        )
        
        self.dec_2 = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size = 5, stride = 2),
            nn.ReLU()
        )       
        
        self.dec_3 = nn.Sequential(
            nn.ConvTranspose2d(16, 1, kernel_size = 8, stride = 2),
            nn.ReLU()
        )        

    def forward(self, x):
        out = self.enc_1(x)
        out = self.enc_2(out)
        out = self.enc_3(out)
        
        out = self.dec_1(out)
        out = self.dec_2(out)
        out = self.dec_3(out)
        return out        
        

In [5]:
model = Autoencoder()

criterion = nn.BCEWithLogitsLoss() #instead of BCELoss() for some fuckin reason
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [6]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #print (images.size())
        
        #forward pass
        outputs = model(images)
        #print(outputs.size())
        loss = criterion(outputs, images)
        
        #backward & optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        """ 
        if (i+1) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        """        
        
torch.save(model.state_dict(), 'toy_autoencoder.pkl')  

In [ ]:
model.eval()
test_image = random.choice(test_dataset)
test_reconst = model((test_image[0].unsqueeze_(0)))
plt.imshow(test_reconst[0][0].detach().numpy()*1000, cmap = 'gray')